# Adaptive PDE discretizations on cartesian grids 
## Volume : GPU accelerated methods
## Part : High dimensional problems
## Chapter : Minimal path for a rolling ball 2

In this notebook, we solve the eikonal equation for a family of optimal control models involving balls, whose analytical models are described in a [previous notebook](RollingBall_Models.ipynb). Let us recall that 
* The state of a ball is determined by three rotation parameters.
* The ball may be placed on line, or on a plane, thus adding one or two parameters to the state space.

We are thus solving sub-Riemannian models in dimension three to five. Depending on the case, they have sub-Riemannian rank (zero,) one or two.

## 0. Importing the required libraries

In [1]:
import sys; sys.path.insert(0,"../..")
#from Miscellaneous import TocTools; print(TocTools.displayTOC('Trailers','GPU'))

In [2]:
from agd import Eikonal
from agd import AutomaticDifferentiation as ad
from agd import Metrics
from agd import LinearParallel as lp
norm_infinity = ad.Optimization.norm_infinity

In [3]:
import numpy as np; xp=np; allclose = np.allclose
from matplotlib import pyplot as plt
np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%5.3g" % x))

In [5]:
def ReloadPackages():
    from Miscellaneous.rreload import rreload
    global Eikonal,ad,cugen,allclose
    Eikonal,ad,cugen,allclose = rreload([Eikonal,ad,cugen,allclose],"..")
    Eikonal.dictIn.default_mode = 'gpu'

### 0.1 Additional configuration

The experiments presented in this notebook require the GPU accelerated solver. 
(Adapting them for the cpu solver would not be too difficult, although computation times would be longer.)

In [67]:
if ad.cupy_generic.cp is None: raise ad.DeliberateNotebookError("cupy module not found")

In [6]:
xp,Eikonal,plt,allclose = [ad.cupy_friendly(e) for e in (xp,Eikonal,plt,allclose)]

Replacing numpy with cupy, set to output 32bit ints and floats by default.
Using cp.asarray(*,dtype=np.float32) as the default caster in ad.array.
Setting dictIn.default_mode = 'gpu' in module agd.Eikonal .
Returning a copy of module matplotlib.pyplot whose functions accept cupy arrays as input.
Setting float32 compatible default values atol=rtol=1e-5 in np.allclose


## 1. Ball models

We consider three dimensional Riemannian and sub-Riemannian models, corresponding to a ball whose rotation can be controlled over three axes, or only two of them. 

### 1.1 The Riemannian ball